In [5]:
import json
import sys
import os
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')
with open("/Users/gisaac/Documents/GitHub/river_flow_notebooks/01104200.json") as f:
    data = json.load(f)

In [2]:
from scraping_functions import HydroScraper

In [7]:
from datetime import datetime
h = HydroScraper(datetime(2023, 1, 1), datetime(2024, 1, 30), "/Users/gisaac/Documents/GitHub/river_flow_notebooks/01104200.json", "weather_asos_final")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=01104200&startDT=2023-01-01&endDT=2024-01-30&parameterCd=00060,00065,00045&siteStatus=all
Request finished


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:112: DtypeWarning: Columns (1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep="\t")
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=OWD&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=2023&month1=1&day1=1&year2=2024&month2=2&day2=1&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4
      station             valid   tmpf   dwpf   relh   feel  sknt  sped  \
0         OWD  2023-01-01 00:53  56.00  54.00  93.32  55.90  3.00  3.45   
1         OWD  2023-01-01 01:53  56.00  53.00  90.29  55.90  3.00  3.45   
2         OWD  2023-01-01 02:53  56.00  53.00  90.29  55.90  4.00  4.60   
3         OWD  2023-01-01 03:53  55.00  53.00  93.29  55.00  0.00  0.00   
4         OWD  2023-01-01 04:01  56.00  53.00  90.29  55.90  0.00  0.00   
...       ...               ...    ...    ...    ...    ...   ...   ...   
13936     OWD  2024-01-31 19:53 

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Scraping completed
LoadJob<project=hydro-earthnet-db, location=EU, id=89a45dd1-fc4b-4c37-9ab1-71767bf441b7>
ASOS data written to BigQuery


In [8]:
h.combine_data()

In [9]:
h.joined_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,station_id,index,agency_cd,site_no,datetime,tz_cd,cfs,66098_00060_cd,height,66099_00065_cd
0,2023-01-01 05:00:00+00:00,3.30,2023-01-01 04:01,55.750000,52.750000,NaN,1005.5,0.0,0.00,29.71,...,OWD,1,USGS,01104200,2023-01-01 05:00:00+00:00,EST,521.0,A,3.93,A
1,2023-01-01 06:00:00+00:00,1.02,2023-01-01 05:39,56.000000,53.000000,NaN,1004.4,200.0,6.90,29.67,...,OWD,13,USGS,01104200,2023-01-01 06:00:00+00:00,EST,521.0,A,3.93,A
2,2023-01-01 07:00:00+00:00,0.76,2023-01-01 06:06,56.000000,53.000000,NaN,1003.5,210.0,6.90,29.66,...,OWD,25,USGS,01104200,2023-01-01 07:00:00+00:00,EST,515.0,A,3.92,A
3,2023-01-01 08:00:00+00:00,0.00,2023-01-01 07:01,55.333333,52.333333,NaN,1004.1,190.0,3.45,29.63,...,OWD,37,USGS,01104200,2023-01-01 08:00:00+00:00,EST,515.0,A,3.92,A
4,2023-01-01 09:00:00+00:00,0.00,2023-01-01 08:20,55.500000,52.000000,NaN,1003.5,260.0,6.90,29.64,...,OWD,49,USGS,01104200,2023-01-01 09:00:00+00:00,EST,515.0,A,3.92,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9467,2024-01-31 00:00:00+00:00,0.00,2024-01-30 23:53,29.000000,19.000000,NaN,1025.0,70.0,5.75,30.27,...,OWD,113698,USGS,1104200,2024-01-31 00:00:00+00:00,EST,863.0,P,4.47,P
9468,2024-01-31 01:00:00+00:00,0.00,2024-01-31 00:53,29.000000,19.000000,NaN,1024.9,60.0,4.60,30.26,...,OWD,113710,USGS,1104200,2024-01-31 01:00:00+00:00,EST,863.0,P,4.47,P
9469,2024-01-31 02:00:00+00:00,0.00,2024-01-31 01:53,29.000000,19.000000,NaN,1024.5,0.0,0.00,30.25,...,OWD,113722,USGS,1104200,2024-01-31 02:00:00+00:00,EST,856.0,P,4.46,P
9470,2024-01-31 03:00:00+00:00,0.00,2024-01-31 02:53,28.000000,19.000000,NaN,1024.4,0.0,0.00,30.25,...,OWD,113734,USGS,1104200,2024-01-31 03:00:00+00:00,EST,863.0,P,4.47,P


In [10]:
h.combine_snotel_with_df()

KeyError: 'snotel'

In [ ]:
h.final_df.to_csv("s11059300_89_24.csv")

In [11]:
import pandas as pd

In [ ]:
!pwd

In [12]:
sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")

In [13]:
h.combine_sentinel(sentinel_df)
h.final_df

KeyError: "None of [Index(['SENSING_TIME', 'BASE_URL'], dtype='object')] are in the [columns]"

In [ ]:
def combine_sentinel(final_df, sentinel_df, tile):
    """ to combine the Sentinel data with the joined ASOS, USGS, and SNOTEL data.
    """
    sentinel_df = sentinel_df[sentinel_df["mgrs_tile"]==tile] 
    sentinel_df = sentinel_df[["sensing_time", "base_url"]]
    sentinel_df["SENSING_TIME"] = pd.to_datetime(sentinel_df["sensing_time"], utc=True, format='mixed').round('60min')
    final_df =final_df.merge(sentinel_df, left_on="hour_updated", right_on="SENSING_TIME", how="left")
    return final_df

fin_df = combine_sentinel(h.final_df, sentinel_df, "11SMT")
fin_df

In [ ]:
fin_df.dropna(subset=["base_url"])

In [ ]:
 sentinel_df_11 = sentinel_df[sentinel_df["mgrs_tile"]=="11SMT"] 

In [ ]:
sentinel_df_11

In [ ]:
find = fin_df

In [ ]:
find.to_csv("final_11059300.csv")